# Experiments 19-22 (640 px) - _Inference on L size_

- **Model:**
    1. `yolov8m` *(Medium)*
    2. `yolov9m` *(Medium)*
    3. `yolov11m` *(Medium)*
    4. `yolov12m` *(Medium)*
- **Dataset:** 3.5m | 90º
- **Sizes:** small & mid
- **Crop Tiles:** 640px
- **Experiments:**
    - Inferencia sobre imagen de `planta grande` (L) con cada modelo _Medium_ (1-4)

### Init

In [1]:
import os
import shutil
import fnmatch
import pandas as pd

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

## Helper Functions

In [3]:
def label_to_str(export: pd.DataFrame) -> list[str]:

    """
    Converts each row of a dataframe into string format

    Parameters:
    - Dataframe with labels (np.dataframe)

    Returns:
    - List with labels in text (list[str])
    """

    lines = []
    for _, row in export.iterrows():
        row_list = list(row)
        row_list[0] = int(row_list[0])
        line = ' '.join(map(str, row_list))
        lines.append(line)

    # Verificar que la cantidad de lineas coincide con la cantidad de labels
    if not len(lines) == len(export):
        raise ValueError("🚨 CUIDADO: el número de etiquetas en el archivo de salida y el dataframe no coinciden.")

    return lines

In [4]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [5]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [6]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [7]:
def copy_directory(source, destination, overwrite=True):
    try:
        if overwrite and os.path.exists(destination):
            shutil.rmtree(destination)  # Remove existing directory
        shutil.copytree(source, destination)
        print(f"Directory '{source}' copied to '{destination}'")
        return True  # Indicate success
    except FileExistsError:
        print(f"Destination '{destination}' exists. Use overwrite=True to replace.")
        return False  # Indicate failure

In [8]:
import pickle

def save_results(results, filename, verbose = True):
    output_file = f"{filename}.pkl"
    with open(output_file, 'wb') as f:
        pickle.dump(results, f)

    if verbose:
      print(f"✅ Results saved as PKL to {output_file}")

def load_results(filename, verbose = True):
    with open(filename, 'rb') as f:
        return pickle.load(f)

    if verbose:
      print(f"✅ Results loaded as PKL from {filename}")

In [9]:
import json

def serialize_results(obj):
    if isinstance(obj, YOLO):
        return str(obj)  # Convert YOLO model object to string representation
    elif hasattr(obj, 'tolist'):  # Check if object has tolist() method (e.g., NumPy arrays, tensors)
        return obj.tolist()
    elif isinstance(obj, (str, int, float, bool, type(None))):
        return obj  # Directly serialize basic types
    else:
        try:
            # Attempt to convert object to dictionary if possible
            return obj.__dict__
        except AttributeError:
            raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

def save_as_json(results, filename="results.json", verbose = True):
    output_file = f"{filename}.json"
    serialized_results = []
    for result in results:
        serialized_result = {
            "path": result.path,
            "names": result.names,
            "boxes": serialize_results(result.boxes),
            "masks": serialize_results(result.masks),
            "probs": serialize_results(result.probs),
            "keypoints": serialize_results(result.keypoints),
            "speed": result.speed  # Assuming speed is already JSON serializable
        }
        serialized_results.append(serialized_result)

    with open(output_file, "w") as f:
        json.dump(serialized_results, f, default=serialize_results, indent=4)

    if verbose:
      print(f"✅ Results saved as JSON to {output_file}")


In [10]:
def save_bboxes(results, filename, verbose = True):
    """
    Convierte los resultados de bounding boxes de YOLO a formato COCO TXT.

    Args:
        results (ultralytics.engine.results.Results): Resultados de la inferencia de YOLO.
        output_file (str): Ruta al archivo TXT de salida.
    """
    output_file = f"{filename}.txt"
    with open(output_file, 'w') as f:
        for result in results:
            if result.boxes:
                boxes = result.boxes.xywhn.tolist() # Obtener bounding boxes normalizados (xywhn)
                classes = result.boxes.cls.tolist() # Obtener clases.
                for box, cls in zip(boxes, classes):
                    x_center, y_center, width, height = box
                    f.write(f"{int(cls)} {x_center} {y_center} {width} {height}\n")

    if verbose:
      print(f"✅ Results saved as TXT to {output_file}")

In [11]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."

    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive
Se realizará la carga de cada tile de la imagen desde la carpeta inference

In [12]:
!rm -rf /content/sample_data

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


A modo de prueba, se cargarán las imagenes pre-procesadas y dividadas en mosaicos.

In [14]:
drive_path = '/content/drive/MyDrive/YOLO/Inference/'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

There are 3 dataset options:


['S', 'M', 'L']

In [15]:
choose_dataset = 3
index = choose_dataset - 1
model = os.listdir(drive_path)[index]
print("Chosen dataset:", model)

Chosen dataset: L


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 2:** is desired if you are goint to work just with one dataset  (avoid downloading unnecessary data from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading any data from the cloud).



In [ ]:
# Option 1 (download all datasets)
!cp -r /content/drive/MyDrive/YOLO/ /content
src_folder = f"/content/YOLO/{model}"

In [16]:
# Option 2 (download just the needed dataset)
cloud_path = f"/content/drive/MyDrive/YOLO/Inference/{model}/"
local_path = f"/content/YOLO/Inference/"
!mkdir "/content/YOLO/"
!mkdir $local_path
!cp -r $cloud_path $local_path
src_folder = f"/content/YOLO/Inference/{model}"

In [ ]:
# Option 3 (don't download it, just open it from the cloud)
src_folder = f"{drive_path}/{model}"

# Carga de modelos entrenados

In [17]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [18]:
models = []

In [19]:
# Load best YOLOv8 stored model
model1 = YOLO("/content/drive/MyDrive/YOLO/models/best8.pt")
models.append(model1)
# Medium | 640px | 1000epoch + early

In [20]:
# Load best YOLOv9 stored model
model2 = YOLO("/content/drive/MyDrive/YOLO/models/best9.pt")
models.append(model2)
# Medium | 640px | 1000epoch + early

In [21]:
# Load best YOLOv11 stored model
model3 = YOLO("/content/drive/MyDrive/YOLO/models/best11.pt")
models.append(model3)
# Medium | 640px | 1000epoch + early

In [22]:
# Load best YOLOv12 stored model
model4 = YOLO("/content/drive/MyDrive/YOLO/models/best12.pt")
models.append(model4)
# Medium | 640px | 1000epoch + early

In [23]:
#models = [model1, model2, model3, model4]
print(f"Se han cargado un total de {len(models)} modelos.")

Se han cargado un total de 4 modelos.


# Inference
Se realizarán las inferencias para cada imagen de forma iterativa (con cada modelo)

In [24]:
experiment="experiment3"
print("Experimento: ", experiment)
print("Source:", src_folder)

Experimento:  experiment3
Source: /content/YOLO/Inference/L


In [25]:
files_paths = os.listdir(src_folder)
files_paths.sort()
total_files = len(files_paths)
print(f'Se detectaron un total de {total_files} imágenes:\n')
files_paths

Se detectaron un total de 45 imágenes:



['tile00x00.DJI_0315.640px.jpg',
 'tile00x01.DJI_0315.640px.jpg',
 'tile00x02.DJI_0315.640px.jpg',
 'tile00x03.DJI_0315.640px.jpg',
 'tile00x04.DJI_0315.640px.jpg',
 'tile00x05.DJI_0315.640px.jpg',
 'tile00x06.DJI_0315.640px.jpg',
 'tile00x07.DJI_0315.640px.jpg',
 'tile00x08.DJI_0315.640px.jpg',
 'tile01x00.DJI_0315.640px.jpg',
 'tile01x01.DJI_0315.640px.jpg',
 'tile01x02.DJI_0315.640px.jpg',
 'tile01x03.DJI_0315.640px.jpg',
 'tile01x04.DJI_0315.640px.jpg',
 'tile01x05.DJI_0315.640px.jpg',
 'tile01x06.DJI_0315.640px.jpg',
 'tile01x07.DJI_0315.640px.jpg',
 'tile01x08.DJI_0315.640px.jpg',
 'tile02x00.DJI_0315.640px.jpg',
 'tile02x01.DJI_0315.640px.jpg',
 'tile02x02.DJI_0315.640px.jpg',
 'tile02x03.DJI_0315.640px.jpg',
 'tile02x04.DJI_0315.640px.jpg',
 'tile02x05.DJI_0315.640px.jpg',
 'tile02x06.DJI_0315.640px.jpg',
 'tile02x07.DJI_0315.640px.jpg',
 'tile02x08.DJI_0315.640px.jpg',
 'tile03x00.DJI_0315.640px.jpg',
 'tile03x01.DJI_0315.640px.jpg',
 'tile03x02.DJI_0315.640px.jpg',
 'tile03x0

In [26]:
import time

def start_stopwatch():
  start_time = time.time()  # Momento de inicio
  return start_time

def end_stopwatch(start_time):
  end_time = time.time()  # Momento de fin
  return end_time - start_time  # Calcula el tiempo de ejecución

In [27]:
def count_plants(model, file_name, verbose=False):
  count: int = 0

  # Realiza inferencia para la imagen
  image = f"{src_folder}/{file_name}"
  results = model(image, save=True, project=project, name=experiment, verbose=verbose)

  # Acceder a las coordenadas de los bounding boxes
  for result in results:
      print(result.verbose()) if verbose else ''
      for box in result.boxes:
          count +=1

  # Se almacena cada inferencia como JSON y PKL (log)
  file = os.path.splitext(file_name)[0]
  save_as = f"/content/{project}/{experiment}/inference.{file}"
  save_results(results, save_as, verbose = verbose)
  save_as_json(results, save_as, verbose = verbose)
  save_bboxes(results, save_as, verbose = verbose)

  return count

In [28]:
import os
import cv2

def count_plants(model, file_name, verbose=False):
  count: int = 0

  # Realiza inferencia para la imagen
  image_path = os.path.join(src_folder, file_name)  # Use os.path.join for platform compatibility

  # Validate image before processing
  if not os.path.exists(image_path):
    print(f"WARNING ⚠️ Image file not found: {image_path}")
    return 0  # Skip if file not found

  try:
    image = cv2.imread(image_path)  # Read image using cv2.imread directly
    if image is None:
      print(f"WARNING ⚠️ Image Read Error {image_path}")
      return 0  # Skip if image could not be read
    results = model(image, save=True, project=project, name=experiment, verbose=verbose)
  except Exception as e:
    print(f"Error processing image {image_path}: {e}")
    return 0  # Skip if any error occurs

  # Acceder a las coordenadas de los bounding boxes
  for result in results:
      print(result.verbose()) if verbose else ''
      for box in result.boxes:
          count +=1

  # Se almacena cada inferencia como JSON y PKL (log)
  file = os.path.splitext(file_name)[0]
  save_as = f"/content/{project}/{experiment}/inference.{file}"
  save_results(results, save_as, verbose = verbose)
  save_as_json(results, save_as, verbose = verbose)
  save_bboxes(results, save_as, verbose = verbose)

  return count

In [29]:
# Ejecución en batch para todos los mosaicos
verbose = False # Activar para incluir más información sobre almacenamiento de predicciones
total_time = []
total_count = []
model = None
plant_count = 0
for count, file_name in enumerate(files_paths):

    print(f"❇️ Procesado {round((count+1)/total_files*100,3):.2f}%\n") if count > 0 else ''
    print(f"INFERENCE {count+1}/{total_files}:\n {file_name}")
    print()
    infer_time = []
    infer_count = []

    for index, model in enumerate(models):

        # Setup del entrenamiento
        project= f"model{index}"
        print(f"Proyecto: {project}")
        timer = start_stopwatch() # Inicia medición de tiempo
        plant_count = count_plants(model, file_name,) # Ejecuta la inferencia y guarda los datos
        measured_time = end_stopwatch(timer) # Termina medición de tiempo
        print(f"  Tiempo de inferencia {(measured_time*1000):.4} ms")
        print(f"  Se detectaron {plant_count} objetos plant-weed.")
        infer_time.append(measured_time) # Acumula el histórico de rendimiento
        infer_count.append(plant_count) # Acumula el histórico del conteo
        print()

    total_time.append(infer_time)
    total_count.append(infer_count)
    print() if verbose else ''

df_count = pd.DataFrame(total_count)
df_time = pd.DataFrame(total_time)
print(f"✅ EJECUCIÓN COMPLETA\n")

count_sums = df_count.sum().values
time_sums = df_time.sum().values

print("Totales por modelo:")
for index, (time_, count_) in enumerate(zip(time_sums, count_sums)):
    print(f"  Modelo {index + 1} ({float(time_):.2f} seg): {int(count_)}")
print()

INFERENCE 1/45:
 tile00x00.DJI_0315.640px.jpg

Proyecto: model0
Results saved to model0/experiment3
  Tiempo de inferencia 4.278e+03 ms
  Se detectaron 17 objetos plant-weed.

Proyecto: model1
Results saved to model1/experiment3
  Tiempo de inferencia 454.1 ms
  Se detectaron 20 objetos plant-weed.

Proyecto: model2
Results saved to model2/experiment3
  Tiempo de inferencia 439.5 ms
  Se detectaron 23 objetos plant-weed.

Proyecto: model3
Results saved to model3/experiment3
  Tiempo de inferencia 409.8 ms
  Se detectaron 48 objetos plant-weed.

❇️ Procesado 4.44%

INFERENCE 2/45:
 tile00x01.DJI_0315.640px.jpg

Proyecto: model0
Results saved to model0/experiment32
  Tiempo de inferencia 49.0 ms
  Se detectaron 17 objetos plant-weed.

Proyecto: model1
Results saved to model1/experiment32
  Tiempo de inferencia 47.75 ms
  Se detectaron 39 objetos plant-weed.

Proyecto: model2
Results saved to model2/experiment32
  Tiempo de inferencia 44.36 ms
  Se detectaron 31 objetos plant-weed.

Proye

In [31]:
count_avg = df_count.mean().values
time_avg = df_time.mean().values

print("Promedios por modelo:")
for index, (time_, count_) in enumerate(zip(time_avg, count_avg)):
    print(f"  Modelo {index + 1} ({float(time_*1000):.3f} ms): {float(count_):.1f} obj/image")
print()

Promedios por modelo:
  Modelo 1 (139.172 ms): 42.9 obj/image
  Modelo 2 (60.178 ms): 57.4 obj/image
  Modelo 3 (58.122 ms): 52.1 obj/image
  Modelo 4 (82.595 ms): 153.6 obj/image



# Almacenamiento de predicciones

In [32]:
# Store experiments
!mkdir /content/drive/MyDrive/Results
for index, model in enumerate(models):
    project= f"model{index}"
    save_on_cloud(source=f'/content/{project}/{experiment}/', destination=f'/content/drive/MyDrive/Results/{project}/{experiment}/')
    print()



✅ Folder copied successfully:
   /content/model0/experiment3/ 
  --> /content/drive/MyDrive/Results/model0/experiment3/

✅ Folder copied successfully:
   /content/model1/experiment3/ 
  --> /content/drive/MyDrive/Results/model1/experiment3/

✅ Folder copied successfully:
   /content/model2/experiment3/ 
  --> /content/drive/MyDrive/Results/model2/experiment3/

✅ Folder copied successfully:
   /content/model3/experiment3/ 
  --> /content/drive/MyDrive/Results/model3/experiment3/



In [33]:
# Save execution time log
filenames = [{
    'file': df_time,
    'name': f"{experiment}.total_time.csv"},
     {'file': df_count,
    'name': f"{experiment}.total_count.csv"}
  ]

for filename in filenames:
  try:
      name = filename['name']
      df = filename['file']
      if df is not None:
        df.to_csv(f"/content/{name}")
      print(f"✅ Execution log saved as CSV to {name}")

      source=f'/content/{name}'
      destination='/content/drive/MyDrive/Results/'
      !cp $source $destination

  except PermissionError:
      print(f"Error: Permission denied. Could not save {name}")
  except OSError as e:
      print(f"Error: An operating system error occurred: {e} when trying to save {name}.")
  except Exception as e:
      print(f"An unexpected error occurred: {e} when trying to save {name}.")

✅ Execution log saved as CSV to experiment3.total_time.csv
✅ Execution log saved as CSV to experiment3.total_count.csv


# Eliminar archivos de inferencia (en caso de errores)

In [ ]:
proceed = False

In [ ]:
# Removes pervious train (CAUTION)
if proceed:
  for index, model in enumerate(models):
      project= f"model{index}"
      source=f'/content/{project}/'
      !rm -rf $source

In [ ]:
if proceed:
  file = f"/content/{experiment}.total_count.csv"
  !rm $file
  file = f"/content/{experiment}.total_time.csv"
  !rm $file